In [1]:
import sys
sys.path.append('../../build')
import libry as ry

import cv2 as cv
from ipywidgets import interact
import numpy as np
import time

In [2]:
C = ry.Config()
C.addFile('../../../scenarios/pandasTable.g')
V = ry.ConfigurationViewer()
cameraFrame = C.frame("camera")

q0 = C.getJointState()
R_gripper = C.frame("R_gripper")
R_gripper.setContact(1)
L_gripper = C.frame("L_gripper")
L_gripper.setContact(1)

V.setConfiguration(C)

In [3]:
#create object to grasp
obj = C.addFrame("Object")
obj.setColor([1, 0, 0])
obj.setShape(ry.ST.ssBox, [0.2, 0.05, 0.1, 0.01])
obj.setPosition([.7, 0, .7])
obj.setMass(1)
obj.setContact(1)

#@interact(x=(-1, 1, 0.1), y = (-1, 1, 0.1), z = (-1, 2, 0.1), a = (0, 1, 0.1))
#def set_position(x, y, z):
#    obj.setPosition([x, y, z])
#    V.setConfiguration(C)

X0 = C.getFrameState()

In [4]:
#S = C.simulation(ry.SimulatorEngine.physx, True)
#S.addSensor("camera")

In [5]:
C.setFrameState(X0)
IK = C.komo_IK(False)
#IK.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['Object', 'R_gripperCenter'], target = [0])
IK.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductYY, frames=['Object', 'R_gripperCenter'], target = [0])
IK.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['Object', 'R_gripperCenter'], target = [1])
IK.addObjective(type=ry.OT.eq, feature=ry.FS.positionRel, frames=['Object','R_gripperCenter'], target=[0 , 0, -.05])
IK.addObjective(type=ry.OT.eq, feature=ry.FS.accumulatedCollisions, frames=['Object','R_finger1'], scale = [1e2])
#IK.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['Object','R_finger1'])
IK.optimize()
C.setFrameState( IK.getConfiguration(0) )
q_below = C.getJointState()

V.setConfiguration(C)

In [6]:
S = C.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [7]:
coll = C.feature(ry.FS.accumulatedCollisions, [])
C.computeCollisions()
print(coll.eval(C))


(array([0.]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))
